<a href="https://colab.research.google.com/github/nyp-sit/iti107-2024S2/blob/main/session-4/contextual_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Embedding

One of the main drawbacks of embeddings such as Word2Vec and GloVE are that they have the same embedding for the same word regardless of its meaning in a particular context. For example, the word `rock` in `The rock concert is being held at national stadium` have a very different meaning in `The naughty boy throws a rock at the dog`.

Contextual embedding such as those produced by transformers (where the modern-day large language are based on) took into account the context of the word, and different embedding is generated for the same word depending on the context.

## Install Hugging Face Transformers library
If you are running this notebook in Google Colab, you will need to install the Hugging Face transformers library as it is not part of the standard environment.

In [2]:
%%capture
!pip install transformers
!pip install datasets

Let's try to generate some embeddings using one of the transformer model `deberta`.

In [3]:
from transformers import AutoModel, AutoTokenizer
# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Load a language model
model = AutoModel.from_pretrained("bert-base-uncased")
# Tokenize the sentence
tokens = tokenizer('The rock concert is being held at national stadium.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 1996, 2600, 4164, 2003, 2108, 2218, 2012, 2120, 3346, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS]
the
rock
concert
is
being
held
at
national
stadium
.
[SEP]


We will pass the tokens through the model to generate embeddings.  We will take the embedding produced by the last layer.

In [5]:
# Process the tokens
embeddings_1 = model(**tokens)[0]
print(embeddings_1)

tensor([[[-0.3166, -0.1838,  0.2260,  ..., -0.2543,  0.7752,  0.0488],
         [-0.1927, -0.6046,  0.2336,  ...,  0.2897,  0.7180, -0.6018],
         [-0.1190, -0.9072,  0.5508,  ...,  0.3140,  0.6360, -0.6950],
         ...,
         [ 1.2431, -0.1377,  0.3431,  ..., -0.6644,  0.0162,  0.3894],
         [ 0.6572, -0.0776, -0.1438,  ...,  0.1538, -0.1535, -0.2956],
         [ 0.3060,  0.2439,  0.1642,  ...,  0.4568, -0.2911, -0.4256]]],
       grad_fn=<NativeLayerNormBackward0>)


**Questions**

1. What is the shape of the embeddings?
2. Why is the shape is such?

Let's try to find the embedding of the token 'rock' used here.

In [6]:
embedding_rock1 = embeddings_1[0][2]
print(embedding_rock1)

tensor([-1.1900e-01, -9.0717e-01,  5.5078e-01,  4.1691e-01, -2.3745e-01,
        -2.2437e-01,  3.8768e-01, -4.3841e-01, -1.0749e-02, -6.7593e-01,
        -5.3246e-01, -4.7773e-01, -9.1051e-02,  3.6918e-01, -6.1276e-01,
         6.7287e-01, -4.3269e-01,  1.9167e-01,  1.3853e-01,  5.3495e-01,
         7.7856e-03,  3.3875e-01,  3.3998e-01,  7.2995e-02,  9.0195e-01,
        -4.2645e-01, -1.7335e-01,  7.3863e-01,  1.1561e-01, -3.8009e-01,
         1.6201e-01,  1.0191e+00,  6.9154e-02,  2.8428e-01, -2.5539e-01,
         7.5791e-02,  5.0260e-01,  3.7820e-01,  6.0168e-01, -3.9929e-01,
        -1.8263e-01, -9.5464e-01,  8.2211e-01,  3.5743e-01, -1.2824e-01,
        -2.9457e-01,  1.7110e-01,  4.7908e-01, -1.9023e-01,  2.2210e-02,
         1.1548e+00,  1.1554e+00,  9.2738e-02, -1.1235e-01,  1.9980e-01,
        -8.0275e-02, -5.3796e-01, -5.1051e-02,  1.8537e-01, -3.0817e-03,
         4.9766e-03,  4.6971e-01,  1.2574e-01, -8.7330e-01,  1.6851e-01,
         6.4237e-01,  7.1010e-01, -3.9240e-01,  2.2

Now write codes to find the embeddings of the word `rock` as used in the sentence `The naughty boy throws a rock at the dog.` and `The boy throws the rock into the drain`.


In [7]:
tokens = tokenizer('The naughty boy throws a rock at the dog.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))
embeddings_2 = model(**tokens)[0]
embedding_rock2 = embeddings_2[0][6]
print(embedding_rock2)

{'input_ids': tensor([[  101,  1996, 20355,  2879, 11618,  1037,  2600,  2012,  1996,  3899,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS]
the
naughty
boy
throws
a
rock
at
the
dog
.
[SEP]
tensor([-2.8811e-01, -3.0404e-01, -4.1673e-01, -2.3028e-01,  5.7508e-01,
        -5.5791e-01, -3.3608e-02,  3.2303e-01,  3.6723e-01,  5.6078e-01,
        -2.7141e-01, -6.5178e-02, -3.7613e-01,  6.3506e-01, -4.3335e-01,
        -1.0045e-01,  1.6953e-02, -2.4991e-01, -6.9872e-02,  7.7598e-01,
         6.4950e-02,  2.0158e-01, -6.0215e-01, -6.5504e-02,  1.2721e+00,
        -2.2570e-01, -1.6607e-01,  1.0569e+00, -4.6342e-01,  1.6123e-02,
        -1.4358e-02,  1.6208e-01,  5.1336e-01, -1.3931e-01, -2.5648e-01,
         5.5053e-02,  4.8340e-01,  3.5856e-01, -2.5234e-01, -6.3437e-01,
         1.8468e-01, -3.8639e-01,  3.1904e-01,  2.4968e-01,  9.1851e-02,
        -1.1393e-01,  8.9152e-01, -2.

In [8]:
tokens = tokenizer('A big rock falls from the slope after heavy rain.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))
embeddings_3 = model(**tokens)[0]
embedding_rock3 = embeddings_3[0][3]
print(embedding_rock3)

{'input_ids': tensor([[ 101, 1037, 2502, 2600, 4212, 2013, 1996, 9663, 2044, 3082, 4542, 1012,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS]
a
big
rock
falls
from
the
slope
after
heavy
rain
.
[SEP]
tensor([ 2.9141e-01, -7.5065e-02, -2.6791e-01, -3.7929e-01,  8.0095e-01,
        -3.4936e-01,  4.5318e-01,  6.9900e-01,  5.7246e-01,  2.1469e-01,
         4.0304e-01, -7.2792e-01, -1.1631e-01,  8.7117e-01, -8.8669e-01,
        -5.1031e-01,  5.6448e-01, -4.1083e-01, -1.2021e-01,  6.3107e-01,
        -2.2309e-01, -6.3625e-02, -4.4825e-01, -4.2715e-01,  9.8619e-01,
        -2.8091e-01,  5.2584e-02,  1.2054e+00, -2.0407e-01,  9.5321e-02,
         1.3667e-01,  6.4009e-01, -2.9576e-01,  6.5244e-01, -1.4647e-02,
         2.7094e-01,  1.0699e+00,  2.4964e-01, -2.8838e-01, -2.8349e-01,
        -3.2069e-01, -2.7276e-01,  2.0257e-01,  5.4540e-01, -2.9399e-01,
        -1.5448e-01,  6.5149e

Let's compute how similar are the embeddings to each other

In [9]:
import torch

cos = torch.nn.CosineSimilarity(dim=0)
similarity1 = cos(embedding_rock1, embedding_rock2)
print(similarity1)

similarity2 = cos(embedding_rock2, embedding_rock3)
print(similarity2)



tensor(0.4528, grad_fn=<SumBackward1>)
tensor(0.7437, grad_fn=<SumBackward1>)


We can see that embedding_rock2 are more similar to embedding_rock3 than with embedding_rock1.

In [20]:
from datasets import load_dataset

# downloaded the datasets.
test_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv'
train_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv'

dataset = load_dataset('csv', data_files=train_data_url, split="train").shuffle().select(range(2000))

In [21]:
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 2000
})

In [13]:
def process_dataset(sample):
    sample['sentiment'] = 0 if sample['sentiment'] == 'negative' else 1
    return sample

dataset = dataset.map(process_dataset)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [22]:
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train'].batch(batch_size=32)
test_dataset = dataset['test'].batch(batch_size=32)

Batching examples:   0%|          | 0/1600 [00:00<?, ? examples/s]

Batching examples:   0%|          | 0/400 [00:00<?, ? examples/s]

## Tokenization

We will now load the DistilBert tokenizer for the pretrained model "distillbert-base-uncased".  This is the same as the other lab exercise.

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Here we will tokenize the text string, and pad the text string to the longest sequence in the batch, and also to truncate the sequence if it exceeds the maximum length allowed by the model (in BERT's case, it is 512).

In [29]:
for data in train_dataset:
    train_encodings = tokenizer(data['review'], padding=True, max_length=512, truncation=True, return_tensors='pt')
    train_labels = data['sentiment']
    outputs = model(**train_encodings)
    print(outputs.shape)

KeyboardInterrupt: 

In [ ]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, padding=True, truncation=True)

In [ ]:
train_encodings[1]

We will create a tensorflow dataset and use it's efficient batching later to obtain the embeddings.

In [ ]:
BATCH_SIZE = 16

In [ ]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size)

Here we instantiate a pretrained model from 'distilbert-base-cased' and specify output_hidden_state=True so that we get the output from each of the attention layers.

## Feature Extraction using (Distil)BERT.

Here we will load the pretrained model for distibert-based-uncased and use it to extract features from the text (i.e. emeddings).

In [ ]:
from transformers import TFAutoModel

model = TFAutoModel.from_pretrained("distilbert-base-uncased",output_hidden_states=True)

The model will produce two outputs: the 1st output `output[0]` is of shape `(16, 512, 768)` which corresponds to the output of the last hidden layer and the second output `output[1]` is a list of 7 outputs of shape `(16, 512, 768)`, corresponding to the output of each of the 6 attention layers and the output. 768 refers to the hidden size.

In [ ]:
def extract_features(dataset):

    embeddings = []
    labels = []

    for encoding, label in dataset:
        output = model(encoding)
        hidden_states = output[1]
        # here we take the output of the second last attention layer as our embeddings.
        # We take the average of the embedding value of 512 tokens (at axis=1) to generate sentence embedding
        sentence_embedding = tf.reduce_mean(hidden_states[-2], axis=1).numpy()
        embeddings.append(sentence_embedding)
        labels.append(label)

    embeddings, labels = np.concatenate(embeddings), np.concatenate(labels)

    return embeddings, labels

In [ ]:
X_train, y_train = extract_features(train_dataset)
X_val, y_val = extract_features(val_dataset)
X_test, y_test = extract_features(test_dataset)

## Train a classifier using the extracted features (embeddings)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:
print(f'train score : {clf.score(X_train, y_train)}')
print(f'validation score : {clf.score(X_val, y_val)}')
print(f'test score : {clf.score(X_test, y_test)}')

We should be getting an validation and accuracy score of around 86% to 87% which is quite good, considering we are training with only 2000 samples!

**Exercise**

1. Modify the code to use the hidden states from a different attention layer as features or take average of hidden states  from few layers as features.
2. Modify the code to use BERT model and see if it performs better than the DistilBERT. For BERT Model, the output of different layers are in `output[2]`